Deep Learning Programming Assignment 2
--------------------------------------
Name:Vishnu Dutt Sharma
Roll No.: 12EC35018

Submission Instructions:
1. Fill your name and roll no in the space provided above.
2. Name your folder in format <Roll No>_<First Name>.
    For example 12CS10001_Rohan
3. Submit a zipped format of the file (.zip only).
4. Submit all your codes. But do not submit any of your datafiles
5. From output files submit only the following 3 files. simOutput.csv, simSummary.csv, analogySolution.csv
6. Place the three files in a folder "output", inside the zip.

In [49]:
import gzip
import os

import pickle
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import random
import csv
import tensorflow as tf

## paths to files. Do not change this
simInputFile = "Q1/word-similarity-dataset"
analogyInputFile = "Q1/word-analogy-dataset"
vectorgzipFile = "Q1/glove.6B.300d.txt.gz"
vectorTxtFile = "Q1/glove.6B.300d.txt"   # If you extract and use the gz file, use this.
analogyTrainPath = "Q1/wordRep/"
simOutputFile = "Q1/simOutput.csv"
simSummaryFile = "Q1/simSummary.csv"
anaSOln = "Q1/analogySolution.csv"
Q4List = "Q4/wordList.csv"




In [50]:
# Similarity Dataset
simDataset = [item.split(" | ") for item in open(simInputFile).read().splitlines()]
# Analogy dataset
analogyDataset = [[stuff.strip() for stuff in item.strip('\n').split('\n')] for item in open(analogyInputFile).read().split('\n\n')]

def vectorExtract(simD = simDataset, anaD = analogyDataset, vect = vectorgzipFile):
    simList = [stuff for item in simD for stuff in item]
    analogyList = [thing for item in anaD for stuff in item[0:6] for thing in stuff.split()]
    simList.extend(analogyList)
    wordList = set(simList)
    print len(wordList)
    wordDict = dict()
    
    vectorFile = gzip.open(vect, 'r')
    for line in vectorFile:
        if line.split()[0].strip() in wordList:
            wordDict[line.split()[0].strip()] = line.split()[1:]
    
    
    vectorFile.close()
    print 'retrieved', len(wordDict.keys())
    return wordDict

# Extracting Vectors from Analogy and Similarity Dataset
validateVectors = vectorExtract()

1221
retrieved 1214


In [51]:
# Dictionary of training pairs for the analogy task
trainDict = dict()
for subDirs in os.listdir(analogyTrainPath):
    for files in os.listdir(analogyTrainPath+subDirs+'/'):
        f = open(analogyTrainPath+subDirs+'/'+files).read().splitlines()
        trainDict[files] = f
print len(trainDict.keys())

19


In [52]:
word2vec = dict()

ind  =  0
with gzip.open(vectorgzipFile, 'r') as f:
    for line in f:
    
        arr = line.split(' ')
        key = (arr[0].strip()).lower()
        val = [float(x) for x in arr[1:]]
        
        word2vec[key] = val
        ind += 1
        
#         if(ind % 10000 == 0):
#             print ind
        
        
f.close()
# word2vec = pickle.load(open('Q1/word2vec.p', 'rb'))

In [53]:

def similarityTask(inputDS = simDataset, outputFile = simOutputFile, summaryFile=simSummaryFile, vectors=validateVectors):
    corr_C = 0
    corr_E = 0
    corr_M = 0
    
    total = 0
    
    mrr_C = []
    mrr_E = []
    mrr_M = []
    
    simFile = open(outputFile, 'w')
    
    for item in inputDS:
        query = np.asarray(vectors[item[0]], dtype=float)
        norm_q = np.linalg.norm(query)
        
        flag = 0
        C = []
        E = []
        M = []
        for option in item[1:]:
            try:
                opt_vec = np.asarray(vectors[option], dtype=float)
            except:
                flag = 1
                break
                
            opt_norm = np.linalg.norm(opt_vec)
            
#             print len(query)
#             print len(opt_vec)
#             print np.inner(query, opt_vec)
            
            cos = np.inner(query, opt_vec)/(norm_q * opt_norm)
            euc = np.linalg.norm(query - opt_vec)
            man = sum(abs(query - opt_vec))
            
            simFile.write(str(total+1) + ',' + item[0] + ',' + option + ',' + 'C' + ',' + str(cos) + '\n')
            simFile.write(str(total+1) + ',' + item[0] + ',' + option + ',' + 'E' + ',' + str(euc) + '\n')
            simFile.write(str(total+1) + ',' + item[0] + ',' + option + ',' + 'M' + ',' + str(man) + '\n')
            
            C.append(cos)
            E.append(euc)
            M.append(man)
        
        if flag:
            flag = 0
            continue
        
#         print C
        ind = np.argmax(C)
        if( ind == 0):
            corr_C += 1
        mrr_C.append(1.0/(1+np.argsort(C)[3]))
        
#         print E
        ind = np.argmin(E)
        if( ind == 0):
            corr_E += 1
        mrr_E.append(1.0/(1+np.argsort(E)[0]))
        
#         print M
        ind = np.argmin(M)
        if( ind == 0):
            corr_M += 1
        mrr_M.append(1.0/(1+np.argsort(M)[0]))
        
#         mrr_C.append(1.0/(1+np.argmin(np.argsort(C))))
#         mrr_E.append(1.0/(1+np.argmin(np.argsort(E))))
#         mrr_M.append(1.0/(1+np.argmin(np.argsort(M))))
    
        total += 1
    
    simFile.close()
    
    sumFile = open(summaryFile, 'w')
    sumFile.write('C'+ ',' + str(corr_C) + ',' + str(total) + ',' + str(sum(mrr_C)/total) + '\n')
    sumFile.write('E'+ ',' + str(corr_E) + ',' + str(total) + ',' + str(sum(mrr_E)/total) + '\n')
    sumFile.write('M'+ ',' + str(corr_M) + ',' + str(total) + ',' + str(sum(mrr_M)/total) + '\n')
    sumFile.close()
    
    print corr_C, corr_E, corr_M
    print sum(mrr_C)/total, sum(mrr_E)/total, sum(mrr_M)/total
    
# similarityTask()

In [54]:
def init_weight(shape):
    return tf.Variable(tf.random_normal(shape, 0, 0.001))
#     return tf.Variable( tf.random_normal(shape, stddev=0.1) )

def forward_prop(X, w1, w2):
    hidden = tf.nn.relu( tf.matmul(X, w1) )

    y_hat = tf.matmul(hidden, w2)
    return y_hat

In [62]:
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer



In [83]:
def analogyTask(inputDS=analogyDataset,outputFile = anaSOln ): # add more arguments if required
    
#     """
#     Output a file, analogySolution.csv with the following entris
#     Query word pair, Correct option, predicted option    
#     """


#     return accuracy #return the accuracy of your model after 5 fold cross validation

# analogyTask()
    RANDOM_SEED = 54
    optn_1 = 1
    optn_2 = 0

    tf.set_random_seed(RANDOM_SEED)


    train_X_raw = []
    train_Y_raw = []

    keys = trainDict.keys()
    
    
    if optn_1:

        for i in range(2000):
            try:
                arr = np.random.randint(19, size = 2)
                word_1 = random.choice(trainDict[keys[arr[0]]]).split('\t')
                word_2 = random.choice(trainDict[keys[arr[1]]]).split('\t')
                train_X_raw += [[1] + word2vec[word_1[0].lower()]+ word2vec[word_1[1].lower()]+ word2vec[word_2[0].lower()]+ word2vec[word_2[1].lower()]]
                train_Y_raw += [[1, 0]]
            except:
                continue

        for i in range(2000):
            try:
                arr = np.random.randint(19)
                word_1 = random.choice(trainDict[keys[arr]]).split('\t')
                word_2 = random.choice(trainDict[keys[arr]]).split('\t')
                train_X_raw += [[1] + word2vec[word_1[0].lower()] + word2vec[word_1[1].lower()] + word2vec[word_2[0].lower()] + word2vec[word_2[1].lower()]]
                train_Y_raw += [[0, 1]]
            except:
                continue

    if optn_2:
        flag = 0
        for item in inputDS:
            query = item[0].split(' ')

            vec_X = []
            for num in range(1,6):
                option = item[num].split(' ')
    #             test_X_raw += [[1] + word2vec[query[0].lower()] + word2vec[query[1].lower()] + word2vec[option[0].lower()] + word2vec[option[1].lower()]]
                try:
                    vec_X += [[1] + word2vec[query[0].lower()] + word2vec[query[1].lower()] + word2vec[option[0].lower()] + word2vec[option[1].lower()]]
                except:
    #                 vec_X += [[0.0] * (300*4+1)]
                    flag = 1
                    break

            if flag:
                flag = 0
                continue

            train_X_raw += vec_X

            zer = [[1, 0]]*5
            zer[ord(item[-1])-97] = [0, 1]
            train_Y_raw +=  zer

#     print len(train_X_raw)
#     print len(train_Y_raw)
#     print len(train_X_raw[0])


    all_X = np.ndarray(shape=(len(train_X_raw), len(train_X_raw[0])))
    all_y = np.ndarray(shape=(len(train_Y_raw), 2))

#     print len(train_X_raw)
#     print len(train_Y_raw)

    index_shuf = range(len(train_Y_raw))
    random.shuffle(index_shuf)
    j = 0
    for i in index_shuf:
        all_X[j] = np.asarray(train_X_raw[i])
        all_y[j] = np.asarray(train_Y_raw[i])
        j += 1

    train_X, test_X, train_y, test_y = train_test_split(all_X, all_y, test_size=0.20, random_state=RANDOM_SEED)


#     x_size = train_X.shape[1]
#     h_size = 300                
#     y_size = train_y.shape[1]

#     X = tf.placeholder("float", shape=[None, x_size])
#     y = tf.placeholder("float", shape=[None, y_size])


#     w_1 = init_weight((x_size, h_size))
#     w_2 = init_weight((h_size, y_size))

#     y_hat = forward_prop(X, w_1, w_2)
#     predict  = tf.nn.softmax(y_hat)

#     cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_hat))
#     updates = tf.train.GradientDescentOptimizer(0.005).minimize(cost)

    
    n_hidden_1 = 300*2  # 1st layer number of features
    n_hidden_2 = 300# 2nd layer number of features
    n_input = train_X.shape[1] # MNIST data input (img shape: 28*28)
    n_classes = train_y.shape[1] # MNIST total classes (0-9 digits)

    
    X = tf.placeholder("float", shape=[None, n_input])
    y = tf.placeholder("float", shape=[None, n_classes])

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    
 
    
        # Construct model
    predict = multilayer_perceptron(X, weights, biases)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=y))
    updates = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    


    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    
    accuracy = 0.0
    
    for epoch in range(5):
        train_X, test_X, train_y, test_y = train_test_split(all_X, all_y, test_size=0.20, random_state=random.randint(1,99))
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 np.argmax(sess.run(predict, feed_dict={X: train_X, y: train_y}), axis=1))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 np.argmax(sess.run(predict, feed_dict={X: test_X, y: test_y}), axis=1))

        accuracy += test_accuracy
        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
    

    accuracy = accuracy/5.0
    
    test_X_raw = []
    test_Y_raw = []

    
    flag = 0
    for item in inputDS:
        query = item[0].split(' ')

        vec_X = []
        for num in range(1,6):
            option = item[num].split(' ')
#             test_X_raw += [[1] + word2vec[query[0].lower()] + word2vec[query[1].lower()] + word2vec[option[0].lower()] + word2vec[option[1].lower()]]
            try:
                vec_X += [[1] + word2vec[query[0].lower()] + word2vec[query[1].lower()] + word2vec[option[0].lower()] + word2vec[option[1].lower()]]
            except:
#                 vec_X += [[0.0] * (300*4+1)]
                flag = 1
                break
        
        if flag:
            flag = 0
            continue
            
        test_X_raw += vec_X

        zer = [[1, 0]]*5
        zer[ord(item[-1])-97] = [0, 1]
        test_Y_raw +=  zer

    test_X = np.asarray(test_X_raw)
    test_y = np.asarray(test_Y_raw)
    # .reshape((len(test_Y_raw),1))

#     print test_X.shape
#     print test_y.shape

    out_file = open(outputFile, 'w')
    
    pos = 0
    neg = 0
    output = sess.run( predict, feed_dict={X: test_X, y: test_y})
#     print 'LEN',len(output)
    for i in range(test_y.shape[0]/5):
        out = np.argmax(output[i*5: (i+1)*5],axis=0)[1]   
#         print output[i*5: (i+1)*5]
#         print out, analogyDataset[i][0],chr(out+97), analogyDataset[i][-1] 
        
        
        out_file.write(analogyDataset[i][0])
        out_file.write(',')
        out_file.write(chr(out+97))
        out_file.write(',')
        out_file.write(analogyDataset[i][-1])
        out_file.write('\n')
        
        if( chr(out+97) == analogyDataset[i][-1]):
            pos += 1
        else:
            neg += 1

    sess.close()
    out_file.close()
            
#     print pos
#     print neg

#     return accuracy

    return 100*float(pos)/(pos+neg)
# analogyTask()


Epoch = 1, train accuracy = 84.89%, test accuracy = 60.30%
Epoch = 2, train accuracy = 90.59%, test accuracy = 79.54%
Epoch = 3, train accuracy = 90.52%, test accuracy = 82.13%
Epoch = 4, train accuracy = 96.11%, test accuracy = 92.22%
Epoch = 5, train accuracy = 91.81%, test accuracy = 89.63%


24.742268041237114

(array([ 0.85701067]), array([ 0.31210075]))

In [56]:
def derivedWordTask(inputFile = Q4List):
    fasttext_dict = dict()
    with open('Q4/fastText_vectors.txt', 'r') as f:
        for line in f:

            arr = line.strip().split(' ')
            key = (arr[0].strip()).lower()
            val = [float(x) for x in arr[1:]]

            fasttext_dict[key] = val

    lazaridou_dict = dict()
    with open('Q4/vector_lazaridou.txt', 'r') as f:
        for line in f:

            arr = line.strip().split('[')
            key = (arr[0].strip()).lower()
            arr = (arr[1].strip()[:-1]).split(',')
            val = [float(x) for x in arr[1:]]

            lazaridou_dict[key] = val

    index = []
    affix = []
    derived = []
    root = []

    csvfile = open(inputFile, 'rb')
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    header = spamreader.next()
    for row in spamreader:
        index.append(int(row[0]))
        affix.append(row[1])
        derived.append(row[2])
        root.append(row[3])
    csvfile.close()

    name = list(set(affix))
    aff_dict = dict(zip(name, range(len(name))))

    h_size  = 300

    train_X_ft = []
    train_Y_ft = []
    for i in range(len(root)):
        try:
            zer = [0]*len(name)
            zer[aff_dict[affix[i]]] = 1
            train_X_ft += [[1]+fasttext_dict[root[i]] + zer]
            train_Y_ft += [fasttext_dict[derived[i]]]
        except:
            continue

    train_X_lz = []
    train_Y_lz = []                      
    for i in range(len(root)):
        try:
            zer = [0]*len(name)
            zer[aff_dict[affix[i]]] = 1
            train_X_lz += [[1]+lazaridou_dict[root[i]] + zer]
            train_Y_lz += [lazaridou_dict[derived[i]]]
        except:
            continue


#     train_X_ft = np.asarray(train_X_ft)
#     train_Y_ft = np.asarray(train_Y_ft)
#     train_X_lz = np.asarray(train_X_lz)
#     train_Y_lz = np.asarray(train_Y_lz)
    
    
    x_size_ft = len(train_X_ft[0])
    h_size_ft = 300                
    y_size_ft = len(train_Y_ft[0])

    x_size_lz = len(train_X_lz[0])
    h_size_lz = 400        
    y_size_lz = len(train_Y_lz[0])

    
    X_ft = tf.placeholder("float", shape=[None, x_size_ft])
    y_ft = tf.placeholder("float", shape=[None, y_size_ft])
    X_lz = tf.placeholder("float", shape=[None, x_size_lz])
    y_lz = tf.placeholder("float", shape=[None, y_size_lz])
    

    w_ft_1 = init_weight((x_size_ft, h_size_ft))
    w_ft_2 = init_weight((h_size_ft, y_size_ft))
    w_lz_1 = init_weight((x_size_lz, h_size_lz))
    w_lz_2 = init_weight((h_size_lz, y_size_lz))

    
    y_hat_ft = forward_prop(X_ft, w_ft_1, w_ft_2)
    predict_ft = y_hat_ft
    y_hat_lz = forward_prop(X_lz, w_lz_1, w_lz_2)
    predict_lz = y_hat_lz



    cost_ft = tf.reduce_mean(tf.nn.l2_loss(y_ft - y_hat_ft))
#     cost_ft = tf.reduce_mean(tf.squared_difference(y_ft, y_hat_ft))
#     cost_ft = tf.reduce_mean(tf.square(y_ft - y_hat_ft))
    updates_ft = tf.train.GradientDescentOptimizer(0.01).minimize(cost_ft)
    
    cost_lz = tf.reduce_mean(tf.nn.l2_loss(y_lz - y_hat_lz))
#     cost_lz = tf.reduce_mean(tf.squared_difference(y_lz, y_hat_lz))
#     cost_lz = tf.reduce_mean(tf.square(y_lz - y_hat_lz))
    updates_lz = tf.train.AdamOptimizer(0.001).minimize(cost_lz)



    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
       
        
    for epoch in range(5):
        train_X, test_X, train_y, test_y = train_test_split(train_X_ft, train_Y_ft, test_size=0.20, random_state=random.randint(1,99))
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates_ft, feed_dict={X_ft: train_X[i: i + 1], y_ft: train_y[i: i + 1]})
            
#         print len(train_y)
#         print len(train_y[0])
#         print np.mean(np.square(train_y - sess.run(predict_ft, feed_dict={X_ft: train_X, y_ft: train_y}))).shape
        
#         train_accuracy = np.mean(np.square(train_y - sess.run(predict_ft, feed_dict={X_ft: train_X, y_ft: train_y})))
#         test_accuracy  = np.mean(np.square(test_y - sess.run(predict_ft, feed_dict={X_ft: test_X, y_ft: test_y}) ))

      
        print "FT: Epoch = ",epoch + 1
#         print("FT: Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
#               % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
    
    for epoch in range(5):
        train_X, test_X, train_y, test_y = train_test_split(train_X_lz, train_Y_lz, test_size=0.20, random_state=random.randint(1,99))
        # Train with each example
        
        for i in range(len(train_X)):
            sess.run(updates_lz, feed_dict={X_lz: train_X[i: i + 1], y_lz: train_y[i: i + 1]})
            
#         train_accuracy = np.mean(np.square(train_y - sess.run(predict_lz, feed_dict={X_lz: train_X, y_lz: train_y})))
#         test_accuracy  = np.mean(np.square(test_y - sess.run(predict_lz, feed_dict={X_lz: test_X, y_lz: test_y}) ))

        print "LZ: Epoch = ",epoch + 1
#         print("LZ: Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
#               % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
        
    
    ft_file = open('Q4/AnsFastText.txt', 'w')
    lz_file = open('Q4/AnsLzaridou.txt', 'w')
    mod_ft_file = open('Q4/AnsModel.txt', 'w')
#     mod_lz_file = open('Q4/AnsModelLZ.txt', 'w')

    ft_c = 0.0
    lz_c = 0.0
    tot = 0
    for i in range(len(root)):
        ft_file.write(derived[i])
        out_1_ft = fasttext_dict[derived[i]]
        for k in out_1_ft:
            ft_file.write(' '+str(k))
        ft_file.write('\n')

        lz_file.write(derived[i])
        out_1_lz = lazaridou_dict[derived[i]]
        for k in out_1_lz:
            lz_file.write(' '+str(k))
        lz_file.write('\n')

        zer = [0]*len(name)
        zer[aff_dict[affix[i]]] = 1
        train_X = [[1]+fasttext_dict[root[i]] + zer]

        mod_ft_file.write(derived[i])

        out_2_ft = sess.run( predict_ft, feed_dict={X_ft: train_X})

        for k in out_2_ft:
            mod_ft_file.write(' '+str(k))
        mod_ft_file.write('\n')

        zer = [0]*len(name)
        zer[aff_dict[affix[i]]] = 1
        train_X = [[1]+lazaridou_dict[root[i]] + zer]


        out_2_lz = sess.run( predict_lz, feed_dict={X_lz: train_X})

        norm1 = np.linalg.norm(np.asarray(out_1_ft))
        norm2 = np.linalg.norm(out_2_ft)
        if(norm1 < 0.001 or norm2 < 0.001):
            cos_ft = 0.0
        else:
            cos_ft = np.inner(np.asarray(out_1_ft), out_2_ft)/(norm1 * norm2)
        
        norm1 = np.linalg.norm(np.asarray(out_1_lz))
        norm2 = np.linalg.norm(out_2_lz)
        if(norm1 < 0.001 or norm2 < 0.001):
            cos_lz = 0.0
        else:
            cos_lz = np.inner(np.asarray(out_1_lz), out_2_lz)/(norm1 * norm2)

        ft_c += cos_ft
        lz_c += cos_lz
        tot += 1


    ft_file.close()
    lz_file.close()
    mod_ft_file.close()

    cosVal1 = ft_c/tot
    cosVal2 = lz_c/tot

    return cosVal1, cosVal2
# derivedWordTask()

In [84]:
def main():
    similarityTask()
    anaSim = analogyTask()
    derCos1,derCos2 = derivedWordTask()
    print anaSim
    print derCos1, derCos2
    
if __name__ == '__main__':
    main()


23 17 17
0.770833333333 0.671296296296 0.662037037037
Epoch = 1, train accuracy = 85.24%, test accuracy = 61.70%
Epoch = 2, train accuracy = 91.54%, test accuracy = 77.70%
Epoch = 3, train accuracy = 92.30%, test accuracy = 85.09%
Epoch = 4, train accuracy = 95.93%, test accuracy = 92.89%
Epoch = 5, train accuracy = 94.52%, test accuracy = 92.75%
FT: Epoch =  1
FT: Epoch =  2
FT: Epoch =  3
FT: Epoch =  4
FT: Epoch =  5
LZ: Epoch =  1
LZ: Epoch =  2
LZ: Epoch =  3
LZ: Epoch =  4
LZ: Epoch =  5
18.5567010309
[ 0.85518842] [ 0.32187445]


In [58]:

# word2vec = pickle.load(open('Q1/word2vec.p', 'rb'))

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


In [30]:
# def derivedWordTask(inputFile = Q4List):
#     fasttext_dict = dict()
#     with open('Q4/fastText_vectors.txt', 'r') as f:
#         for line in f:

#             arr = line.strip().split(' ')
#             key = (arr[0].strip()).lower()
#             val = [float(x) for x in arr[1:]]

#             fasttext_dict[key] = val

#     lazaridou_dict = dict()
#     with open('Q4/vector_lazaridou.txt', 'r') as f:
#         for line in f:

#             arr = line.strip().split('[')
#             key = (arr[0].strip()).lower()
#             arr = (arr[1].strip()[:-1]).split(',')
#             val = [float(x) for x in arr[1:]]

#             lazaridou_dict[key] = val

#     index = []
#     affix = []
#     derived = []
#     root = []

#     csvfile = open(Q4List, 'rb')
#     spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#     header = spamreader.next()
#     for row in spamreader:
#         index.append(int(row[0]))
#         affix.append(row[1])
#         derived.append(row[2])
#         root.append(row[3])
#     csvfile.close()

#     name = list(set(affix))
#     aff_dict = dict(zip(name, range(len(name))))

#     h_size  = 300

#     train_X_raw_ft = []
#     train_Y_raw_ft = []
#     train_X_ft = []
#     train_Y_ft = []
#     for i in range(len(root)):
#         try:
#             zer = [0]*len(name)
#             zer[aff_dict[affix[i]]] = 1
#             train_X_raw_ft += [[1]+ fasttext_dict[root[i]] + zer]
#             train_Y_raw_ft += [fasttext_dict[derived[i]]]
#         except:
#             continue

#     index_shuf = range(len(train_Y_raw_ft))
#     random.shuffle(index_shuf)
#     for i in index_shuf:
#         train_X_ft.append(train_X_raw_ft[i])
#         train_Y_ft.append(train_Y_raw_ft[i])

#     train_X_raw_lz = []
#     train_Y_raw_lz = []
#     train_X_lz = []
#     train_Y_lz = []                      
#     for i in range(len(root)):
#         try:
#             zer = [0]*len(name)
#             zer[aff_dict[affix[i]]] = 1
#             train_X_raw_lz += [[1]+ lazaridou_dict[root[i]] + zer]
#             train_Y_raw_lz += [lazaridou_dict[derived[i]]]
#         except:
#             continue

#     index_shuf = range(len(train_Y_raw_lz))
#     random.shuffle(index_shuf)
#     for i in index_shuf:
#         train_X_lz.append(train_X_raw_lz[i])
#         train_Y_lz.append(train_Y_raw_lz[i])

#     kf = KFold(n_splits=5)
#     for train_index, test_index in kf.split(train_X_ft):
#     #         print len(train_index)
#     #         print len(test_index)
#             X_train, X_test = np.asarray(train_X_ft)[train_index], np.asarray(train_X_ft)[test_index]
#             Y_train, Y_test = np.asarray(train_Y_ft)[train_index], np.asarray(train_Y_ft)[test_index]
#     #         print X_train[1:10]
#     #         print X_test[1:10]


#             mlp_ft = MLPRegressor(solver='lbfgs', hidden_layer_sizes=300, max_iter=150, shuffle=True, random_state=1, activation='relu')
#             mlp_ft.fit(X_train, Y_train)
#             score = mlp_ft.score(X_test, Y_test)
# #             print 'Score ft: ',score


#     for train_index, test_index in kf.split(train_X_lz):
#     #         print len(train_index)
#     #         print len(test_index)
#             X_train, X_test = np.asarray(train_X_lz)[train_index], np.asarray(train_X_lz)[test_index]
#             Y_train, Y_test = np.asarray(train_Y_lz)[train_index], np.asarray(train_Y_lz)[test_index]
#     #         print X_train[1:10]
#     #         print X_test[1:10]


#             mlp_lz = MLPRegressor(solver='lbfgs', hidden_layer_sizes=450, max_iter=150, shuffle=True, random_state=1, activation='relu')
#             mlp_lz.fit(X_train, Y_train)
#             score = mlp_lz.score(X_test, Y_test)
# #             print 'Score lz: ',score



#     ft_file = open('Q4/AnsFastText.txt', 'w')
#     lz_file = open('Q4/AnsLzaridou.txt', 'w')
#     mod_ft_file = open('Q4/AnsModel.txt', 'w')
# #     mod_lz_file = open('Q4/AnsModelLZ.txt', 'w')

#     ft_c = 0.0
#     lz_c = 0.0
#     tot = 0
#     for i in range(len(root)):
#         ft_file.write(derived[i])
#         out_1_ft = fasttext_dict[derived[i]]
#         for k in out_1_ft:
#             ft_file.write(' '+str(k))
#         ft_file.write('\n')

#         lz_file.write(derived[i])
#         out_1_lz = lazaridou_dict[derived[i]]
#         for k in out_1_lz:
#             lz_file.write(' '+str(k))
#         lz_file.write('\n')

#         zer = [0]*len(name)
#         zer[aff_dict[affix[i]]] = 1
#         train_X = [[1]+ fasttext_dict[root[i]] + zer]

#         mod_ft_file.write(derived[i])

#         out_2_ft = mlp_ft.predict(train_X)

#         for k in out_2_ft:
#             mod_ft_file.write(' '+str(k))
#         mod_ft_file.write('\n')

#         zer = [0]*len(name)
#         zer[aff_dict[affix[i]]] = 1
#         train_X = [[1]+ lazaridou_dict[root[i]] + zer]


#         out_2_lz = mlp_lz.predict(train_X)

#         cos_ft = np.inner(np.asarray(out_1_ft), out_2_ft)/(np.linalg.norm(np.asarray(out_1_ft)) * np.linalg.norm(out_2_ft))
#         cos_lz = np.inner(np.asarray(out_1_lz), out_2_lz)/(np.linalg.norm(np.asarray(out_1_lz)) * np.linalg.norm(out_2_lz))

#         ft_c += cos_ft
#         lz_c += cos_lz
#         tot += 1


#     ft_file.close()
#     lz_file.close()
#     mod_ft_file.close()

#     cosVal1 = ft_c/tot
#     cosVal2 = lz_c/tot

#     return cosVal1, cosVal2
# # derivedWordTask()